In [212]:
from langgraph.graph import StateGraph, START, END 
from typing_extensions import TypedDict
from langgraph.types import Command

In [213]:
class State(TypedDict):
    transfer_reason: str
    
    
graph_builder = StateGraph(State)

In [ ]:
from typing import Literal


def triage_node(state: State) -> Command[Literal["account_support", "tech_support"]]:
    # Jump to another node or graph
    # When you update state in the same time, you can use command, not conditional edge
    return Command(goto="account_support", update={"transfer_reason": "The user wants to change password."})
    
def tech_support(state: State):
    return {}

def account_support(state: State):
    print("account support running ...")
    return {}


In [215]:
graph_builder.add_node("triage_node", triage_node)
graph_builder.add_node("tech_support", tech_support)
graph_builder.add_node("account_support", account_support)

graph_builder.add_edge(START, "triage_node")
graph_builder.add_edge("tech_support", END)
graph_builder.add_edge("account_support", END)

In [217]:
graph = graph_builder.compile()
# graph
print(graph.get_graph().draw_ascii())

                 +-----------+                  
                 | __start__ |                  
                 +-----------+                  
                        *                       
                        *                       
                        *                       
                +-------------+                 
                | triage_node |                 
                +-------------+                 
                ..            ...               
              ..                 ..             
            ..                     ..           
+-----------------+           +--------------+  
| account_support |           | tech_support |  
+-----------------+           +--------------+  
                **            ***               
                  **        **                  
                    **    **                    
                  +---------+                   
                  | __end__ |                   
                  +-

In [209]:
print(graph.invoke({}))


account support running ...
{'transfer_reason': 'The user wants to change password.'}
